In [ ]:
!pip install -qU langchain sentence_transformers
!pip install -qU langchain-huggingface langchain-community
!pip install -qU faiss-cpu
!pip install --upgrade openai
!pip install -qU tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7

In [ ]:
from tqdm import tqdm
import numpy as np
import json
import faiss
import json
import os
import pandas as pd
import re
from google.colab import drive, files
from typing import Any, List, Literal, Optional, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
import statistics
from scipy import stats

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Caminho para a pasta dos arquivos
folder_path = '/content/drive/Shareddrives/RAG Receita/Scraping Normas/'

# Dicionário para armazenar o conteúdo dos arquivos
data = {}

# Itera sobre os arquivos na pasta
for file in os.listdir(folder_path):
    if file.endswith('.txt'):  # Verifica se o arquivo é um .txt
        file_path = os.path.join(folder_path, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Adiciona ao dicionário usando o nome do arquivo (sem extensão) como chave
        key = os.path.splitext(file)[0]
        data[key] = content

# Caminho para salvar o JSON
output_path = '/content/drive/Shareddrives/RAG Receita/normas.json'

# Salva o dicionário como JSON
with open(output_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"JSON gerado com sucesso em: {output_path}")

JSON gerado com sucesso em: /content/drive/Shareddrives/RAG Receita/normas.json


In [ ]:
json_normas_path = '/content/drive/Shareddrives/RAG Receita/normas.json'

with open(json_normas_path, 'r') as file:
    normas_json = json.load(file)

In [ ]:
delimiters = [
    "CAPÍTULO",  # Para capturar capítulos com maiúsculas (formato oficial).
    "Capítulo",  # Para capturar capítulos com maiúscula inicial (pode variar entre fontes).
    "Seção",     # Para seções (geralmente em legislações extensas).
    "Subseção",  # Para subseções (subdivisões de seções).
    "Art. ",     # Para artigos principais.
    "§",         # Para parágrafos.
    "Inciso",    # Para incisos (nem sempre está explícito, mas aparece em alguns textos).
    "a)",        # Para alíneas (listas dentro de incisos ou artigos).
    "b)",        # Continuações de alíneas.
    "Item",      # Para itens (detalhamentos adicionais).
    "Livro",     # Para divisões maiores como livros em códigos.
    "Título",    # Para títulos (grandes divisões, ex: "Título I - Das Disposições Gerais").
    "I -", "II -", "III -", "IV -", "V -", "VI -", "VII -", "VIII -", "IX -", "X -", # divisão de romanos
    "\n\n" # Quebra de parágrafo
]

text_splitter = RecursiveCharacterTextSplitter(
    separators=delimiters,
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100  # Sobreposição entre chunks para contexto
)

chunks_normas = []

# Percorre cada item no JSON
for filename, doc in normas_json.items():
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # Divide o documento em chunks
    chunks = text_splitter.split_text(doc)

    # Adiciona o nome do documento a cada chunk
    for chunk in chunks:
        # Formata cada chunk com o nome do arquivo e o conteúdo
        chunk_with_name = f"{doc_name}: {chunk}"
        chunks_normas.append(chunk_with_name)

In [ ]:
def sliding_window_split(text, max_tokens=2048, stride=1024):
    """
    Divide o texto em chunks usando uma janela deslizante (sliding window).

    Args:
        text (str): O texto a ser dividido.
        max_tokens (int): Tamanho máximo de cada janela em tokens/caracteres.
        stride (int): Quantidade de avanço da janela. Menor que max_tokens para gerar sobreposição.

    Returns:
        List[str]: Lista de chunks divididos pelo sliding window.
    """
    chunks = []
    start = 0

    # Continua enquanto o início da janela for menor que o tamanho do texto
    while start < len(text):
        # Final da janela
        end = start + max_tokens
        chunk = text[start:end]
        chunks.append(chunk)

        # Avança com o stride
        start += stride

    return chunks

# Aplicação no seu caso
chunks_normas_sliding_window = []

for filename, doc in normas_json.items():
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # Divide o texto usando a sliding window
    chunks = sliding_window_split(doc, max_tokens=8192, stride=4096)

    # Adiciona o nome do documento a cada chunk
    for chunk in chunks:
        chunk_with_name = f"{doc_name}: {chunk}"
        chunks_normas_sliding_window.append(chunk_with_name)

In [ ]:
import tiktoken
model_name = "text-embedding-3-small"
tokenizer = tiktoken.encoding_for_model(model_name)

In [ ]:
class HierachicalRecursiveCharacterTextSplitter(RecursiveCharacterTextSplitter):

    def __init__(
        self,
        separators: Optional[List[str]] = None,
        keep_separator: Union[bool, Literal["start", "end"]] = True,
        is_separator_regex: bool = False,
        apply_chunk_size: Optional[int] = 0,
        **kwargs: Any,
    ) -> None:
        """Create a new TextSplitter."""
        super().__init__(keep_separator=keep_separator, **kwargs)
        self._separators = separators or ["\n\n", "\n", " ", ""]
        self._is_separator_regex = is_separator_regex
        self._apply_chunk_size = apply_chunk_size



    def _split_text_with_regex(
        self, text: str, separator: str
    ) -> List[str]:
        # Now that we have the separator, split the text

        separators = None

        if separator:
            if self._keep_separator:
                try:
                    # The parentheses in the pattern keep the delimiters in the result.
                    _splits = re.split(f"({separator})", text)
                    splits = (
                        ([_splits[i] + _splits[i + 1] for i in range(0, len(_splits) - 1, 2)])
                        if self._keep_separator == "end"
                        else ([_splits[i] + _splits[i + 1] for i in range(1, len(_splits), 2)])
                    )
                    if len(_splits) % 2 == 0:
                        splits += _splits[-1:]
                    splits = (
                        (splits + [_splits[-1]])
                        if self._keep_separator == "end"
                        else ([_splits[0]] + splits)
                    )

                    separators = ["preamble"] + [_splits[i].strip("\n").strip() for i in range(1, len(_splits), 2)]
                except IndexError as e:
                    print(e)
                    print(_splits)
            else:
                splits = re.split(separator, text)
        else:
            splits = list(text)
        return [s for s in splits if s != ""], separators



    def _split_text(self, text: str, separators: List[str], steps_to_apply_chunk_size: int) -> List[str]:

        # print(separators)
        # print(steps_to_apply_chunk_size)
        # print(self._length_function(text))

        """Split incoming text and return chunks."""
        final_chunks = {}
        # Get appropriate separator to use
        separator = separators[-1]
        new_separators = []
        for i, _s in enumerate(separators):
            _separator = _s if self._is_separator_regex else re.escape(_s)
            if _s == "":
                separator = _s
                break
            if re.search(_separator, text):
                separator = _s
                new_separators = separators[i + 1 :]
                break

        _separator = separator if self._is_separator_regex else re.escape(separator)

        if steps_to_apply_chunk_size > 0 or self._length_function(text) > self._chunk_size:
            splits, separators_text = self._split_text_with_regex(text, _separator)

            # Does not merge small chunks to respect hierarchy
            if not new_separators:
                final_chunks = dict(zip(separators_text, splits))
            else:
                for i, s in enumerate(splits):
                    # print(f"new_separators={new_separators}, len(new_separators)={len(new_separators)}, len(separators)={len(separators)}")
                    other_info = self._split_text(s, new_separators, steps_to_apply_chunk_size - (len(separators) - len(new_separators)))
                    final_chunks[separators_text[i]] = other_info
        else:
            final_chunks = text

        return final_chunks


    def split_text(self, text: str) -> List[str]:
        """Split the input text into smaller chunks based on predefined separators.

        Args:
            text (str): The input text to be split.

        Returns:
            List[str]: A list of text chunks obtained after splitting.
        """
        return self._split_text(text, self._separators, self._apply_chunk_size)

spliting_hierarchy=[
    "\nLIVRO [IVX]+",
    "\nTÍTULO [IVX]+",
    "\nCAPÍTULO [IVX]+|Capítulo [IVX]+|\nCAPÍTULO ÚNICO",
    "\nSeção [IVX]+",
    "\nSubseção .+",
    # "\nArt\. \d+[º\. ]*|\n.\s+Art\. \d+[º\. ]*|\nArtigo único\.",
    "\nArt\. \d+[º\. ]*|\n.\s+Art\. \d+[º\. ]*",
    "\n§ \d+[º\. ]*",
    "\n[IVX]+[-\s]*",
]

hierarchical_text_splitter = HierachicalRecursiveCharacterTextSplitter(
    separators=spliting_hierarchy,
    is_separator_regex=True,
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100,  # Sobreposição entre chunks para contexto
    apply_chunk_size=6 # Separator index to start applying the chunk size check.
)

In [ ]:
def flatten_passages_hierarchy(passages_dictionary, current_path="", passages=[]):

    for item, value in passages_dictionary.items():
        item_path = current_path

        if item == "preamble":
            if type(value) == dict:
                flatten_passages_hierarchy(value, current_path, passages)

        if item_path != "":
            item_path += "_" + item
        else:
            item_path = item

        if type(value) == dict:
            flatten_passages_hierarchy(value, item_path, passages)
        else:
            passages.append({'path': item_path,
                             'passage': value})

def contar_tokens(chunk):
    tokens = tokenizer.encode(chunk)  # Codifica o texto e retorna a lista de tokens
    num_tokens = len(tokens)  # Conta o número de tokens
    return num_tokens

def passages_statistics(passages_list):
    num_tokens_por_chunk = [contar_tokens(passage) for passage in passages_list]

    desc_stats = stats.describe(num_tokens_por_chunk)

    mediana_tokens = statistics.median(num_tokens_por_chunk)
    max_token_passage = np.argmax(num_tokens_por_chunk)

    return({'max_tokens': desc_stats.minmax[1],
            'min_tokens': desc_stats.minmax[0],
            'mean_tokens': desc_stats.mean,
            'std_tokens': np.sqrt(desc_stats.variance),
            'skewness_tokens': desc_stats.skewness,
            'kurtosis_tokens': desc_stats.kurtosis,
            'median_tokens': statistics.median(num_tokens_por_chunk),
            'max_token_passage': np.argmax(num_tokens_por_chunk)})

applied_splitter = []
passages_referred_docs = []

for filename, doc in tqdm(normas_json.items(), desc="Processing all referred legal documents"):
    # print(f"Processing file: {filename}")

    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # First try applying the hierarchical splitter
    chunks = hierarchical_text_splitter.split_text(doc)
    passages = []
    current_path = ""

    flatten_passages_hierarchy(chunks, current_path, passages)

    # print(chunks.keys())

    total_statistics = passages_statistics([passage['passage'] for passage in passages])

    # print(total_statistics)

    #
    # Check if there are indications the hierarchical splitter did not work
    # verifying if there is any chunk longer than the maximum size.
    #

    if total_statistics['max_tokens'] > 1000:
        # For such document, apply a regular text splitter.

        # print(f">>> Applying regular text splitter in file {filename}.")

        chunks = text_splitter.split_text(doc)
        applied_splitter.append({'filename': filename,
                                 'splitter': 'text'})

        for passage in chunks:
            # Formata cada chunk com o nome do arquivo e o conteúdo
            passage_with_path = "{}: {}".format(doc_name, passage)
            passages_referred_docs.append(passage_with_path)
    else:
        applied_splitter.append({'filename': filename,
                                 'splitter': 'hierarchical'})

        for passage in passages:
            # Formata cada chunk com o nome do arquivo e o conteúdo
            passage_with_path = "{}: {}".format(doc_name + "_" + passage['path'], passage['passage'])
            passages_referred_docs.append(passage_with_path)

Processing all referred legal documents:   0%|          | 1/489 [00:00<01:35,  5.12it/s]/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:1114: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:1114: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
Processing all referred legal documents: 100%|██████████| 489/489 [00:04<00:00, 100.79it/s]


In [ ]:
print(f'Total chunked normas slidingwindow: {len(chunks_normas_sliding_window)}')
print(f'Total chunked normas: {len(chunks_normas)}')
print(f'Total chunked normas hierarchical: {len(passages_referred_docs)}')

Total chunked normas slidingwindow: 4351
Total chunked normas: 24241
Total chunked normas hierarchical: 24030


In [ ]:
import pickle

with open('/content/drive/Shareddrives/RAG Receita/Chunks/chunks_normas_sliding_window.pkl', "wb") as f:
    pickle.dump(chunks_normas_sliding_window, f)

with open('/content/drive/Shareddrives/RAG Receita/Chunks/chunks_normas_recursive.pkl', "wb") as f:
    pickle.dump(chunks_normas, f)

with open('/content/drive/Shareddrives/RAG Receita/Chunks/chunks_normas_hierarchical.pkl', "wb") as f:
    pickle.dump(passages_referred_docs, f)

In [ ]:
# Define o limite máximo de tokens permitido
max_tokens = 8192  # Limite do modelo

def verificar_tamanho_chunks(chunks, max_tokens, tokenizer):
    """
    Verifica o tamanho dos chunks em tokens e imprime aqueles que excedem o limite.

    Args:
        chunks (list): Lista de chunks para verificar.
        max_tokens (int): Limite máximo de tokens permitido.
        tokenizer: Tokenizer usado para contar os tokens.
    """
    for i, chunk in enumerate(chunks):
        num_tokens = len(tokenizer.encode(chunk))
        if num_tokens > max_tokens:
            print(f"Chunk {i} excede o limite de tokens ({num_tokens} tokens):")
            print(chunk[:700])  # Exibe os primeiros 300 caracteres para referência
            print("-" * 50)

In [ ]:
verificar_tamanho_chunks(chunks_normas, max_tokens, tokenizer)

Chunk 11182 excede o limite de tokens (8633 tokens):
Lei nº 11.437: Seção 6 Chefe FCE 1.03 46 Assessor Técnico Especializado FCE 4.03 49 Assessor Técnico Especializado FCE 4.02 17 Assessor Técnico Especializado FCE 4.01 DIRETORIA DE GESTÃO ESTRATÉGICA 1 Diretor FCE 1.15 Coordenação-Geral 5 Coordenador-Geral FCE 1.13 Coordenação 1 Coordenador CCE 1.10 Coordenação 7 Coordenador FCE 1.10 1 Coordenador de Projeto CCE 3.10 1 Assessor Técnico Especializado FCE 4.10 Divisão 5 Chefe FCE 1.07 1 Assessor Técnico Especializado FCE 4.05 DIRETORIA DE GESTÃO DE PESSOAS 1 Diretor CCE 1.15 Coordenação-Geral 1 Coordenador-Geral CCE 1.13 Coordenação-Geral 4 Coordenador-Geral FCE 1.13 1 Gerente de Projeto FCE 3.13 Coordenação 1 Coordenador CCE 1.10 Coordenação 1
--------------------------------------------------
Chunk 12438 excede o limite de tokens (65182 tokens):
Instrução Normativa RFB nº 256: Art. 65. Fica formalmente revogada, sem interrupção de sua força normativa, a Instrução Normativa SRF nº 60

In [ ]:
verificar_tamanho_chunks(passages_referred_docs, max_tokens, tokenizer)

Chunk 11146 excede o limite de tokens (8633 tokens):
Lei nº 11.437: Seção 6 Chefe FCE 1.03 46 Assessor Técnico Especializado FCE 4.03 49 Assessor Técnico Especializado FCE 4.02 17 Assessor Técnico Especializado FCE 4.01 DIRETORIA DE GESTÃO ESTRATÉGICA 1 Diretor FCE 1.15 Coordenação-Geral 5 Coordenador-Geral FCE 1.13 Coordenação 1 Coordenador CCE 1.10 Coordenação 7 Coordenador FCE 1.10 1 Coordenador de Projeto CCE 3.10 1 Assessor Técnico Especializado FCE 4.10 Divisão 5 Chefe FCE 1.07 1 Assessor Técnico Especializado FCE 4.05 DIRETORIA DE GESTÃO DE PESSOAS 1 Diretor CCE 1.15 Coordenação-Geral 1 Coordenador-Geral CCE 1.13 Coordenação-Geral 4 Coordenador-Geral FCE 1.13 1 Gerente de Projeto FCE 3.13 Coordenação 1 Coordenador CCE 1.10 Coordenação 1
--------------------------------------------------
Chunk 12362 excede o limite de tokens (65182 tokens):
Instrução Normativa RFB nº 256: Art. 65. Fica formalmente revogada, sem interrupção de sua força normativa, a Instrução Normativa SRF nº 60

In [ ]:
verificar_tamanho_chunks(chunks_normas_sliding_window, max_tokens, tokenizer)

In [ ]:
import openai

openai.api_key =
# Função para obter embeddings da OpenAI
def get_openai_embeddings(text):
    try:
        # Requisição para a API da OpenAI
        response = openai.embeddings.create(
            model="text-embedding-3-small",  # Modelo de embeddings mais leve
            input=text
        )

        # Extraindo embedding da resposta
        embedding = response.data[0].embedding
        return np.array(embedding)

    except:
      return None

# Gera embeddings para todos os chunks
embeddings = [
    embedding for chunk in tqdm(passages_referred_docs, desc="Gerando embeddings")
    if (embedding := get_openai_embeddings(chunk)) is not None
]

embeddings = np.array(embeddings).astype('float32')
# Cria um índice FAISS
d = embeddings.shape[1]  # Dimensão dos embeddings
index = faiss.IndexFlatL2(d)  # Usando o índice L2 (distância euclidiana)
index.add(embeddings)  # Adiciona os embeddings ao índice
# Define o caminho completo para salvar o índice no Google Drive
faiss_index_file = '/content/drive/Shareddrives/RAG Receita/FAISS/faiss_index_normas_hierarchical.index'
# Salva o índice no caminho especificado
faiss.write_index(index, faiss_index_file)
print(f"Índice FAISS salvo em: {faiss_index_file}")

Gerando embeddings: 100%|██████████| 24030/24030 [2:12:04<00:00,  3.03it/s]


Índice FAISS salvo em: /content/drive/Shareddrives/RAG Receita/FAISS/faiss_index_normas_hierarchical.index


In [ ]:
embeddings2 = [embedding for chunk in tqdm(chunks_normas, desc="Gerando embeddings")
    if (embedding := get_openai_embeddings(chunk)) is not None
]
embeddings2 = np.array(embeddings2).astype('float32')

d2 = embeddings2.shape[1]  # Dimensão dos embeddings
index2 = faiss.IndexFlatL2(d2)  # Usando o índice L2 (distância euclidiana)
index2.add(embeddings2)  # Adiciona os embeddings ao índice
faiss_index_file_normas_recursive_splitter = '/content/drive/Shareddrives/RAG Receita/FAISS/faiss_index_normas_recursive_splitter.index'
faiss.write_index(index2, faiss_index_file_normas_recursive_splitter)
print(f"Índice FAISS salvo em: {faiss_index_file_normas_recursive_splitter}")


Gerando embeddings: 100%|██████████| 24241/24241 [2:29:05<00:00,  2.71it/s]


Índice FAISS salvo em: /content/drive/Shareddrives/RAG Receita/FAISS/faiss_index_normas_recursive_splitter.index


In [ ]:
embeddings3 = [embedding for chunk in tqdm(chunks_normas_sliding_window, desc="Gerando embeddings")
    if (embedding := get_openai_embeddings(chunk)) is not None
]
# Converte os embeddings para float32
embeddings3 = np.array(embeddings3).astype('float32')
d3 = embeddings3.shape[1]  # Dimensão dos embeddings
index3 = faiss.IndexFlatL2(d3)  # Usando o índice L2 (distância euclidiana)
index3.add(embeddings3)  # Adiciona os embeddings ao índice
faiss_index_file_normas_sliding_window = '/content/drive/Shareddrives/RAG Receita/FAISS/faiss_index_normas_sliding_window.index'
faiss.write_index(index3, faiss_index_file_normas_sliding_window)
print(f"Índice FAISS salvo em: {faiss_index_file_normas_sliding_window}")

Gerando embeddings: 100%|██████████| 4351/4351 [28:32<00:00,  2.54it/s]


Índice FAISS salvo em: /content/drive/Shareddrives/RAG Receita/FAISS/faiss_index_normas_sliding_window.index
